In [54]:
using GLM
using TimeSeries, DataFrames
using XLSX
using StatsBase
using PrettyTables


In [326]:
data = DataFrame(XLSX.readtable("assets_data_03.24.xlsx", "Returns";infer_eltypes=true))
data = TimeArray(data,timestamp=:Date)



615×10 TimeArray{Any, 2, Date, Matrix{Any}} 1973-01-31 to 2024-03-31
┌────────────┬────────────┬─────────────┬──────────────┬──────────────┬─────────
│            │ ACWI_Net   │ USDPLN      │ ACWI_Net_PLN │ CPI_USA      │ CPI_PL ⋯
├────────────┼────────────┼─────────────┼──────────────┼──────────────┼─────────
│ 1973-01-31 │    missing │     missing │      missing │   0.00235344 │     mi ⋯
│ 1973-02-28 │    missing │     missing │      missing │   0.00704374 │     mi ⋯
│ 1973-03-31 │    missing │     missing │      missing │   0.00932044 │     mi ⋯
│ 1973-04-30 │    missing │     missing │      missing │   0.00692989 │     mi ⋯
│ 1973-05-31 │    missing │     missing │      missing │   0.00688219 │     mi ⋯
│ 1973-06-30 │    missing │     missing │      missing │   0.00683515 │     mi ⋯
│ 1973-07-31 │    missing │     missing │      missing │   0.00225755 │     mi ⋯
│ 1973-08-31 │    missing │     missing │      missing │    0.0180626 │     mi ⋯
│     ⋮      │     ⋮      │      ⋮      

In [40]:
colnames(data)

8-element Vector{Symbol}:
 :ACWI_Net
 :USDPLN
 :ACWI_Net_PLN
 :CPI_USA
 :CPI_PLN
 :TBSP
 :WIBOR3m
 :TenYPL

In [237]:
function returns_summarystats(data::TimeArray,t)
    names = colnames(data)
    returns = transpose(values(data))
    n_assets = size(returns)[1]

    stats = [ Dict(
        :mean => mean(returns[i,:]) * t, 
        :std => std(returns[i,:]) * t^0.5,
        :median => median(returns[i,:] * t),
        :skewness => skewness(returns[i,:]),
        :kurtosis => kurtosis(returns[i,:]),
        :autocor => autocor(returns[i,:],[1])[1],
        :p25th => percentile(returns[i,:],25) * t,
        :p75th => percentile(returns[i,:],75) * t,
        :min => minimum(returns[i,:]) * t,
        :max => maximum(returns[i,:]) * t,
        ) for i in 1:n_assets ]

    short_stats = pretty_table(stats, backend = Val(:html), row_labels = names)
    return short_stats
end



returns_summarystats (generic function with 1 method)

In [238]:
short = Float64.(from(data[:TBSP, :WIBOR3m, :TenYPL],Date(2007,01,01)))

207×3 TimeArray{Float64, 2, Date, Matrix{Float64}} 2007-01-31 to 2024-03-31
┌────────────┬─────────────┬────────────┬────────────┐
│            │ TBSP        │ WIBOR3m    │ TenYPL     │
├────────────┼─────────────┼────────────┼────────────┤
│ 2007-01-31 │     0.00626 │ 0.00349167 │     0.0043 │
│ 2007-02-28 │ 0.000149067 │ 0.00351667 │ 0.00435333 │
│ 2007-03-31 │  0.00510726 │ 0.00353333 │ 0.00434083 │
│ 2007-04-30 │ 0.000692007 │ 0.00366667 │ 0.00443917 │
│ 2007-05-31 │  0.00362559 │     0.0037 │ 0.00443917 │
│ 2007-06-30 │ -0.00847508 │   0.003925 │ 0.00470083 │
│ 2007-07-31 │  0.00471553 │ 0.00400833 │ 0.00466583 │
│ 2007-08-31 │ -0.00094856 │ 0.00419167 │ 0.00482417 │
│     ⋮      │      ⋮      │     ⋮      │     ⋮      │
│ 2023-09-30 │  0.00622425 │ 0.00480833 │ 0.00491583 │
│ 2023-10-31 │   0.0057372 │ 0.00471667 │ 0.00471167 │
│ 2023-11-30 │  0.00315821 │ 0.00485833 │   0.004585 │
│ 2023-12-31 │   0.0171423 │     0.0049 │  0.0043475 │
│ 2024-01-31 │   0.0045132 │ 0.00488333 │   

In [239]:
returns_summarystats(short,12)

,std,min,max,mean,skewness,p25th,autocor,kurtosis,p75th,median
TBSP,0.0428527,-0.502584,0.827558,0.0403455,0.342799,-0.0282004,0.0553435,5.80245,0.104502,0.0448318
WIBOR3m,0.00580118,0.0021,0.0751,0.0337908,0.271898,0.0172,0.988687,-1.03315,0.04915,0.0274
TenYPL,0.00476751,0.01149,0.08337,0.042673,-0.144031,0.02954,0.977263,-1.17253,0.057915,0.04218


In [199]:
varls = to(Float64.(from(data[:WIBOR3m, :TenYPL],Date(2003,01,01))),Date(2006,12,31))
returns_summarystats(varls,12)

,std,min,max,mean,skewness,p25th,autocor,kurtosis,p75th,median
WIBOR3m,0.0026271,0.0413,0.07,0.0532854,0.168329,0.04475,0.937251,-1.21056,0.05985,0.0542
TenYPL,0.00246801,0.04622,0.07497,0.0581775,0.642365,0.05172,0.912227,-0.865177,0.0661475,0.055605


In [200]:
varls

48×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 2003-01-31 to 2006-12-31
┌────────────┬────────────┬────────────┐
│            │ WIBOR3m    │ TenYPL     │
├────────────┼────────────┼────────────┤
│ 2003-01-31 │ 0.00539167 │ 0.00477833 │
│ 2003-02-28 │   0.005225 │  0.0046275 │
│ 2003-03-31 │ 0.00498333 │   0.004565 │
│ 2003-04-30 │ 0.00474167 │ 0.00441583 │
│ 2003-05-31 │     0.0045 │ 0.00421583 │
│ 2003-06-30 │ 0.00443333 │  0.0042525 │
│ 2003-07-31 │ 0.00436667 │ 0.00472417 │
│ 2003-08-31 │ 0.00428333 │ 0.00471167 │
│     ⋮      │     ⋮      │     ⋮      │
│ 2006-06-30 │ 0.00350833 │  0.0047375 │
│ 2006-07-31 │ 0.00348333 │  0.0046675 │
│ 2006-08-31 │ 0.00349167 │ 0.00466167 │
│ 2006-09-30 │ 0.00351667 │ 0.00459167 │
│ 2006-10-31 │ 0.00350833 │  0.0044175 │
│ 2006-11-30 │     0.0035 │ 0.00431083 │
│ 2006-12-31 │     0.0035 │  0.0043525 │
└────────────┴────────────┴────────────┘
                         33 rows omitted

In [278]:
any(ismissing.(varls))

1×2 TimeArray{Bool, 2, Date, BitMatrix} 2006-12-31 to 2006-12-31
┌────────────┬─────────┬────────┐
│            │ WIBOR3m │ TenYPL │
├────────────┼─────────┼────────┤
│ 2006-12-31 │   false │  false │
└────────────┴─────────┴────────┘

In [282]:
varls[1:end-1]

47×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 2003-01-31 to 2006-11-30
┌────────────┬────────────┬────────────┐
│            │ WIBOR3m    │ TenYPL     │
├────────────┼────────────┼────────────┤
│ 2003-01-31 │ 0.00539167 │ 0.00477833 │
│ 2003-02-28 │   0.005225 │  0.0046275 │
│ 2003-03-31 │ 0.00498333 │   0.004565 │
│ 2003-04-30 │ 0.00474167 │ 0.00441583 │
│ 2003-05-31 │     0.0045 │ 0.00421583 │
│ 2003-06-30 │ 0.00443333 │  0.0042525 │
│ 2003-07-31 │ 0.00436667 │ 0.00472417 │
│ 2003-08-31 │ 0.00428333 │ 0.00471167 │
│     ⋮      │     ⋮      │     ⋮      │
│ 2006-05-31 │ 0.00345833 │  0.0044425 │
│ 2006-06-30 │ 0.00350833 │  0.0047375 │
│ 2006-07-31 │ 0.00348333 │  0.0046675 │
│ 2006-08-31 │ 0.00349167 │ 0.00466167 │
│ 2006-09-30 │ 0.00351667 │ 0.00459167 │
│ 2006-10-31 │ 0.00350833 │  0.0044175 │
│ 2006-11-30 │     0.0035 │ 0.00431083 │
└────────────┴────────────┴────────────┘
                         32 rows omitted

In [286]:
diff_varls = varls[2:end] .- varls[1:end-1]

TypeError: TypeError: non-boolean (Missing) used in boolean context

In [287]:
ols = lm(@formula(TBSP ~ WIBOR3m + TenYPL), short[:TBSP, :WIBOR3m, :TenYPL])

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

TBSP ~ 1 + WIBOR3m + TenYPL

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)   0.00787004  0.00266261   2.96    0.0035   0.00262028   0.0131198
WIBOR3m       4.73141     1.18903      3.98    <1e-04   2.38705      7.07577
TenYPL       -5.01425     1.44683     -3.47    0.0006  -7.86691     -2.16159
──────────────────────────────────────────────────────────────────────────────

In [152]:
println(round(r2(ols),digits=4))


0.0726


In [163]:
res = residuals(ols)

207-element Vector{Float64}:
  0.003430716591018009
 -0.0025310752522496593
  0.002285585345042713
 -0.0022674570120775547
  0.0005084084603820815
 -0.011344760884167953
  0.0012760621939868766
 -0.004461530558817827
  0.0027820739511106307
  0.0011500434712708452
  ⋮
  0.005338244277345139
 -0.009133260284113345
  0.00025321597553191674
 -0.0008238605338106754
 -0.004708281211956773
  0.007887752722751981
 -0.004624845680767025
 -0.011182006298309195
 -0.00807717870245278

In [164]:
deviance(ols)

0.029236942219919678

In [169]:
(sum(res.^2)/(207-3))

0.00014331834421529255

In [196]:
predicted = predict(ols,varls)
display(mean(predicted)*12)
display(std(predicted)*12^0.5)

0.05483928570743096

0.010535360864781951

In [259]:
random_res = mean(rand(res,(10000,48)),dims=1)

1×48 Matrix{Float64}:
 -3.40154e-5  7.98157e-7  0.000125823  2.92536e-5  …  0.000164779  6.23259e-5

In [262]:
backfilled = predicted .+ random_res[1]
display(mean(backfilled)*12)
display(std(backfilled)*12^0.5)

0.054431100620797424

0.010535360864781951

In [263]:
backfilled = TimeArray(timestamp(varls),backfilled,[:TBSP])

48×1 TimeArray{Float64, 1, Date, Vector{Float64}} 2003-01-31 to 2006-12-31
┌────────────┬─────────────┐
│            │ TBSP        │
├────────────┼─────────────┤
│ 2003-01-31 │  0.00938647 │
│ 2003-02-28 │  0.00935422 │
│ 2003-03-31 │  0.00852418 │
│ 2003-04-30 │  0.00812872 │
│ 2003-05-31 │  0.00798814 │
│ 2003-06-30 │  0.00748886 │
│ 2003-07-31 │  0.00480838 │
│ 2003-08-31 │  0.00447677 │
│     ⋮      │      ⋮      │
│ 2006-06-30 │ 0.000680391 │
│ 2006-07-31 │ 0.000913103 │
│ 2006-08-31 │ 0.000981781 │
│ 2006-09-30 │  0.00145106 │
│ 2006-10-31 │  0.00228495 │
│ 2006-11-30 │  0.00278038 │
│ 2006-12-31 │  0.00257145 │
└────────────┴─────────────┘
             33 rows omitted

In [264]:
ls = merge(backfilled, varls)

48×3 TimeArray{Float64, 2, Date, Matrix{Float64}} 2003-01-31 to 2006-12-31
┌────────────┬─────────────┬────────────┬────────────┐
│            │ TBSP        │ WIBOR3m    │ TenYPL     │
├────────────┼─────────────┼────────────┼────────────┤
│ 2003-01-31 │  0.00938647 │ 0.00539167 │ 0.00477833 │
│ 2003-02-28 │  0.00935422 │   0.005225 │  0.0046275 │
│ 2003-03-31 │  0.00852418 │ 0.00498333 │   0.004565 │
│ 2003-04-30 │  0.00812872 │ 0.00474167 │ 0.00441583 │
│ 2003-05-31 │  0.00798814 │     0.0045 │ 0.00421583 │
│ 2003-06-30 │  0.00748886 │ 0.00443333 │  0.0042525 │
│ 2003-07-31 │  0.00480838 │ 0.00436667 │ 0.00472417 │
│ 2003-08-31 │  0.00447677 │ 0.00428333 │ 0.00471167 │
│     ⋮      │      ⋮      │     ⋮      │     ⋮      │
│ 2006-06-30 │ 0.000680391 │ 0.00350833 │  0.0047375 │
│ 2006-07-31 │ 0.000913103 │ 0.00348333 │  0.0046675 │
│ 2006-08-31 │ 0.000981781 │ 0.00349167 │ 0.00466167 │
│ 2006-09-30 │  0.00145106 │ 0.00351667 │ 0.00459167 │
│ 2006-10-31 │  0.00228495 │ 0.00350833 │  0.

In [265]:
CSV.write("tbsp_backfilled.csv",ls)

"tbsp_backfilled.csv"

In [328]:
shortd = Float64.(from(data[:TBSP, :dWIBOR3m, :dTenYPL],Date(2007,01,01)))

207×3 TimeArray{Float64, 2, Date, Matrix{Float64}} 2007-01-31 to 2024-03-31
┌────────────┬─────────────┬──────────┬──────────┐
│            │ TBSP        │ dWIBOR3m │ dTenYPL  │
├────────────┼─────────────┼──────────┼──────────┤
│ 2007-01-31 │     0.00626 │  -0.0001 │ -0.00063 │
│ 2007-02-28 │ 0.000149067 │   0.0003 │  0.00064 │
│ 2007-03-31 │  0.00510726 │   0.0002 │ -0.00015 │
│ 2007-04-30 │ 0.000692007 │   0.0016 │  0.00118 │
│ 2007-05-31 │  0.00362559 │   0.0004 │      0.0 │
│ 2007-06-30 │ -0.00847508 │   0.0027 │  0.00314 │
│ 2007-07-31 │  0.00471553 │    0.001 │ -0.00042 │
│ 2007-08-31 │ -0.00094856 │   0.0022 │   0.0019 │
│     ⋮      │      ⋮      │    ⋮     │    ⋮     │
│ 2023-09-30 │  0.00622425 │  -0.0088 │  0.00329 │
│ 2023-10-31 │   0.0057372 │  -0.0011 │ -0.00245 │
│ 2023-11-30 │  0.00315821 │   0.0017 │ -0.00152 │
│ 2023-12-31 │   0.0171423 │   0.0005 │ -0.00285 │
│ 2024-01-31 │   0.0045132 │  -0.0002 │   9.0e-5 │
│ 2024-02-29 │ -0.00240331 │      0.0 │  0.00086 │
│ 2024

In [329]:
olsd = lm(@formula(TBSP ~ dWIBOR3m + dTenYPL), shortd)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

TBSP ~ 1 + dWIBOR3m + dTenYPL

Coefficients:
─────────────────────────────────────────────────────────────────────────────────
                   Coef.   Std. Error       t  Pr(>|t|)    Lower 95%    Upper 95%
─────────────────────────────────────────────────────────────────────────────────
(Intercept)   0.00346296  0.000230418   15.03    <1e-34   0.00300865   0.00391726
dWIBOR3m     -0.835608    0.103688      -8.06    <1e-13  -1.04005     -0.631171
dTenYPL      -3.26993     0.0739324    -44.23    <1e-99  -3.4157      -3.12416
─────────────────────────────────────────────────────────────────────────────────

In [330]:
r2(olsd)

0.9289694570840676

In [333]:
vardls = to(Float64.(from(data[:dWIBOR3m, :dTenYPL],Date(2003,02,01))),Date(2006,12,31))
returns_summarystats(vardls,12)

,std,min,max,mean,skewness,p25th,autocor,kurtosis,p75th,median
dWIBOR3m,0.00757375,-0.0648,0.0744,-0.00579574,0.936545,-0.021,0.479925,2.07982,0.0018,-0.0048
dTenYPL,0.0121559,-0.09552,0.12468,-0.00130468,0.437535,-0.02172,0.0740282,0.717512,0.02088,-0.00588


In [334]:
predictedd = predict(olsd,vardls)
display(mean(predictedd)*12)
display(std(predictedd)*12^0.5)

0.050664694874785296

0.04350609268871154

In [335]:
predictedd

47-element Vector{Union{Missing, Float64}}:
  0.011052751399214019
  0.0083386712729715
  0.011739400134342434
  0.013734058408800414
  0.0026926754821313343
 -0.01437636745667285
  0.004789056761768258
 -0.012414334697508274
 -0.03569240121221049
 -0.004166893857709727
  ⋮
 -0.0020232761327380775
 -0.002001451988527866
 -0.008613963929931015
  0.006460383834074508
  0.0036082932297641424
  0.005959018922689272
  0.010380676693786142
  0.007732032099833799
  0.0018279930377693046

In [336]:
res = residuals(olsd)

207-element Vector{Float64}:
  0.0006534234385229907
 -0.0009704533168588005
  0.001320935508712469
  0.002424540861044975
  0.0004968710023352872
  0.00058569170309156
  0.0007148075577032217
  0.003639688845070375
  0.0011546612105229155
 -0.0021040380642500314
  ⋮
  0.0013210354772392463
  0.002368484560918753
  0.006166015574676678
 -0.006656255524975235
 -0.0038545150589394655
  0.004777806539335044
  0.0011774175418032074
 -0.003054128666302041
  0.0009070292338371217

In [337]:
mean(rand(res,(10000,48)),dims=1)

1×48 Matrix{Float64}:
 -1.75078e-5  2.03757e-5  -1.45367e-5  …  -1.18314e-5  1.05926e-5  1.81259e-5

In [338]:
backfilledd = predictedd .+ mean(rand(res,(10000,48)),dims=1)
display(mean(backfilledd)*12)
display(std(backfilledd)*12^0.5)
backfilledd[:,1]

0.05076962652647489

0.043050496637368416

47-element Vector{Float64}:
  0.011035265102862526
  0.008321184976620008
  0.011721913837990941
  0.013716572112448921
  0.0026751891857798417
 -0.014393853753024342
  0.004771570465416765
 -0.012431820993859767
 -0.03570988750856199
 -0.004184380154061219
  ⋮
 -0.00204076242908957
 -0.0020189382848793585
 -0.008631450226282507
  0.006442897537723016
  0.00359080693341265
  0.00594153262633778
  0.010363190397434649
  0.0077145458034823065
  0.001810506741417812

In [339]:
backfilled = TimeArray(timestamp(vardls),backfilledd[:,1],[:TBSP])

47×1 TimeArray{Float64, 1, Date, Vector{Float64}} 2003-02-28 to 2006-12-31
┌────────────┬─────────────┐
│            │ TBSP        │
├────────────┼─────────────┤
│ 2003-02-28 │   0.0110353 │
│ 2003-03-31 │  0.00832118 │
│ 2003-04-30 │   0.0117219 │
│ 2003-05-31 │   0.0137166 │
│ 2003-06-30 │  0.00267519 │
│ 2003-07-31 │  -0.0143939 │
│ 2003-08-31 │  0.00477157 │
│ 2003-09-30 │  -0.0124318 │
│     ⋮      │      ⋮      │
│ 2006-06-30 │ -0.00863145 │
│ 2006-07-31 │   0.0064429 │
│ 2006-08-31 │  0.00359081 │
│ 2006-09-30 │  0.00594153 │
│ 2006-10-31 │   0.0103632 │
│ 2006-11-30 │  0.00771455 │
│ 2006-12-31 │  0.00181051 │
└────────────┴─────────────┘
             32 rows omitted

In [340]:
CSV.write("tbsp_backfilledd.csv",backfilled)

"tbsp_backfilledd.csv"